<a href="https://colab.research.google.com/github/ish66726-a11y/colab-notebooks/blob/main/VaR%2CCDO%2C%E9%87%91%E5%88%A9%E3%83%A2%E3%83%87%E3%83%AB%2CVasicek%E3%83%A2%E3%83%87%E3%83%AB%2C%E9%95%B7%E6%9C%9F%E6%8A%95%E8%B3%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vasicek（ワセチェック）モデル

##11-1 Vasicek（ワセチェック）モデルを使った債券価格の算出  
Monte Carlo によるゼロクーポン債のプライシング

---

## 1. ゼロクーポン債の基本

額面（Face Value）を $X$、満期を $T$ とすると  
**金利が一定** の場合の現在価値は

$
P(0,T) = X e^{-rT}
$

例：  
- 額面 $1000  
- 金利 $r=0.10$  
- 満期 $T=2$  

なら

$
P(0,2)=1000 e^{-0.1\times 2}
$

---

## 2. 金利がランダムに動く場合

以前の章で学んだとおり、金利 $r_t$ は実際には **常に変動する**。

そこで金利に対して **Vasicek モデル**を使う。

---

## 3. Vasicek モデル（金利の確率微分方程式）

$
dr_t = \kappa(\theta - r_t)\,dt + \sigma\, dW_t
$

ここで：

| 変数 | 意味 |
|------|------|
| $\kappa$ | 平均回帰速度（Mean reversion speed） |
| $\theta$ | 長期平均金利 |
| $\sigma$ | ボラティリティ |
| $W_t$ | Wiener 過程（標準ブラウン運動） |

**重要点：**

- 金利は **平均 $\theta$ に戻ろうとする**
- その強さが $\kappa$
- ノイズ（不確実性）の大きさが $\sigma$

---

## 4. 金利が確率的に変動する場合の債券価格

本来の現在価値は

$
P(0,T) = \mathbb{E}\left[ e^{-\int_0^T r_t\, dt} \right]
$

ただし積分は金利パス $r_t$ に依存する。  
解析的に厳しいので **Monte Carlo 法**を使う。

---

## 5. Monte Carlo 法による債券価格の考え方

1. **Vasicek SDE を用いて金利パスを多数生成する**
   - $r_t^{(1)}, r_t^{(2)}, \dots , r_t^{(N)}$

2. 各パスで割引因子を計算する：

$
D^{(i)} = e^{- \int_0^T r_t^{(i)} dt }
$

3. その平均を取る：

$
P(0,T) = X \cdot \frac{1}{N} \sum_{i=1}^N D^{(i)}
$

---

## 6. なぜ Monte Carlo を使うのか？

- 金利がランダムなので解析的に解くのが困難
- Vasicek は SDE（確率微分方程式）なので数値解が必要
- Monte Carlo は複雑なモデルでも扱える
- シミュレーション回数を増やすと精度が上がる

---

## 7. 次に実装する内容

次章ではいよいよ **Python で Vasicek を使って債券価格をシミュレーション計算**する。

実装では次のステップを行う：

1. Vasicek モデルで金利パスを多数生成  
2. 各パスで $\int_0^T r_t dt$ を数値積分  
3. 割引因子を計算  
4. 平均を取って債券価格を算出



##11-2 Vasicek（ワセチェック）モデルを用いた債券価格の Monte Carlo 実装  

---

## 1. モデルの目的

- **金利 $r_t$ は変動するため、将来の金利パスを多数生成し、債券価格を計算する必要がある**
- Vasicek モデルを用いて金利プロセスをシミュレーションし、
- 生成した金利パスを使って、ゼロクーポン債の割引現在価値を求める

---

## 2. シミュレーション設定

- **シミュレーション回数**：$N_\text{sim} = 1000$  
- **1パスあたりの金利点数**：$N_\text{points} = 200$  
- 各パスは Vasicek モデルで生成  
- 完成形データは「1000 × 201」の行列（最初の時点を含むため）

---

## 3. Vasicek モデルの離散化式

Vasicek の連続 SDE：

$$
dr_t = \kappa(\theta - r_t)\,dt + \sigma\, dW_t
$$

これを時間刻み $\Delta t$ で離散化すると：

$$
\Delta r_t = \kappa(\theta - r_t)\Delta t + \sigma \sqrt{\Delta t} \; Z
$$

ここで $Z \sim N(0,1)$

---

## 4. Bond Pricing Monte Carlo の流れ

1. 初期金利 $r_0$ をセット  
2. Vasicek モデルで金利パスを $N_\text{sim}$ 回生成  
3. 各パスについて割引因子を計算  
4. 平均して現在価値を得る

---

## 5. 実装コード（ワセチェック金利シミュレーション）

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- シミュレーション設定 ---
N_simulations = 1000      # 金利パスの数
N_points = 200            # 1パスあたりの時点数


# --- Bond Pricing Monte Carlo 関数 ---
def vasicek_monte_carlo(X, r0, kappa, theta, sigma, T=1.0):
    """
    X     : 債券の額面（例：1000）
    r0    : 初期金利
    kappa : 平均回帰速度
    theta : 長期平均金利
    sigma : ボラティリティ
    T     : 満期（年）
    """

    dt = T / float(N_points)
    results = []

    # --- 金利パスを N_simulations 本生成 ---
    for _ in range(N_simulations):
        rates = [r0]

        for _ in range(N_points):
            dr = kappa * (theta - rates[-1]) * dt \
                 + sigma * np.sqrt(dt) * np.random.normal()
            rates.append(rates[-1] + dr)

        results.append(rates)

    return results
```

---

## 6. メイン処理（シミュレーションの実行と可視化）

```python
if __name__ == "__main__":
    # パラメータ（例）
    X = 1000       # 額面
    r0 = 0.01      # 初期金利
    kappa = 0.3
    theta = 0.05
    sigma = 0.02

    # Monte Carlo 実行
    result = vasicek_monte_carlo(X, r0, kappa, theta, sigma)

    # pandas DataFrame に変換
    df = pd.DataFrame(result)
    print(df)

    # プロットのために転置
    df_T = df.T

    # プロット
    df_T.plot(legend=False)
    plt.title("Vasicek Interest Rate Simulations")
    plt.xlabel("Time Step")
    plt.ylabel("Interest Rate")
    plt.show()
```

---

## 7. 出力されるシミュレーションのイメージ

- 行：金利パス（シミュレーション番号）
- 列：時間ステップ
- 転置後（プロット用）：
  - 縦：金利
  - 横：時間

複数の金利パスが広がりながら **平均 $\theta$ の周りに回帰する様子** が見える。

---

## 8. 次に行うこと

次章では：

- 生成した金利パスを使って  
  $$P(0,T) = X \cdot e^{-\int_0^T r_t\, dt}$$  
  を数値的に評価し、
- **ゼロクーポン債価格を Monte Carlo で算出**します。




##11-3 Vasicek（ワセチェック）モデルによる債券価格の計算（続編）

前回は Vasicek モデルを使って **未来の金利パスを大量生成する** ところまでを実装しました。  
今回はこの続きとして：

1. 生成した金利パスを用いて  
2. **ゼロクーポン債の価格を Monte Carlo により算出する**  

という部分をまとめます。

---

## 1. 生成した金利パスを使って何をするのか？

ゼロクーポン債の価格は、以下のように **割引因子の積分** を用いて表される：

$$
P(0,T) = X \cdot \mathbb{E}\left[\exp\left(-\int_0^T r_t\, dt\right)\right]
$$

ここで：

- $X$：額面（例：1000ドル）
- $r_t$：Vasicek モデルで生成した金利パス
- $\int_0^T r_t dt$：金利の積分（面積）

積分は **矩形近似（リーマン和）**で計算する：

$$
\int_0^T r_t dt \approx \sum_{i=0}^{N-1} r_i \, \Delta t
$$

---

## 2. 債券価格に必要なステップ

1. Vasicek モデルで金利パスを多数生成  
2. 각金利パスごとに  
   $$
   I = \sum r_i \Delta t
   $$
   を計算  
3. さらに  
   $$
   e^{-I}
   $$
   を計算  
4. 全シミュレーションで平均し  
   $$
   P(0,T) = X \cdot \text{mean}(e^{-I})
   $$
   を求める

---

## 3. 実装（Vasicek シミュレーション＋債券価格計算）

```python
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- シミュレーション設定 ---
N_simulations = 1000     # 金利パス数
N_points = 200           # 金利パスの点数


def vasicek_monte_carlo(X, r0, kappa, theta, sigma, T=1.0):
    """
    X     : 債券の額面
    r0    : 初期金利
    kappa : 平均回帰速度
    theta : 長期平均
    sigma : ボラティリティ
    T     : 満期（年）
    """
    dt = T / float(N_points)
    results = []

    # --- 金利パスを多数生成 ---
    for _ in range(N_simulations):
        rates = [r0]

        for _ in range(N_points):
            dr = kappa * (theta - rates[-1]) * dt \
                 + sigma * np.sqrt(dt) * np.random.normal()
            rates.append(rates[-1] + dr)

        results.append(rates)

    return np.array(results)  # shape: (N_sim, N_points+1)


if __name__ == "__main__":
    # パラメータ例
    X = 1000
    r0 = 0.05
    kappa = 0.3
    theta = 0.09
    sigma = 0.02
    T = 1.0

    # --- 金利シミュレーション ---
    rates = vasicek_monte_carlo(X, r0, kappa, theta, sigma, T)

    # DataFrame に変換（任意）
    df = pd.DataFrame(rates)
    print(df.head())

    # --- 債券価格の計算 ---
    dt = T / float(N_points)

    # 積分（リーマン和）
    integral_sum = rates.sum(axis=1) * dt

    # 割引因子の計算
    discount_factors = np.exp(-integral_sum)

    # 平均を取る（Monte Carlo）
    bond_price = X * np.mean(discount_factors)

    print(f"Bond Price (Monte Carlo): ${bond_price:,.2f}")

    # --- プロット ---
    df.T.plot(legend=False)
    plt.title("Vasicek Interest Rate Simulations")
    plt.xlabel("Time Step")
    plt.ylabel("Interest Rate")
    plt.show()
```

---

## 4. 計算結果の例

- 初期金利が高い → 割引が強くなる → **債券価格は低い**  
- 初期金利が低い → 割引が弱い → **債券価格は高い**

例：

| r0 | theta | Bond Price |
|----|--------|-------------|
| 0.10 | 0.09 | 約 \$570 |
| 0.01 | 0.03 | 約 \$880 |

---

## 5. まとめ

- Vasicek モデルは **平均回帰**する金利を表現できるため債券価格モデルに適している  
- Monte Carlo により  
  $$
  \mathbb{E}[e^{-\int r_t dt}]
  $$
  を直接推定できる  
- 実装は比較的シンプルで、拡張もしやすい


